In [43]:
import pandas as pd
import os
import glob


import numpy as np
from scipy.stats import hypergeom
import matplotlib.pyplot as plt


In [78]:
ICD_FILES_PATH = "/cs/labs/michall/roeizucker/10krun/results"
ESNG_CONVERSION_FILE = "/sci/nosnap/michall/roeizucker/pwas_hub/gProfiler_hsapiens_5-20-2024_11-26-08AM.csv"
path = '/sci/nosnap/michall/roeizucker/jupyter_notebooks/PWAS_Hub'
ICD_TRANSALTOR_PATH = "/cs/labs/michall/roeizucker/10krun/ICD10_tree_filterd2.csv"
GLOBAL_THREASHOLD = 0.5
GENETICS_THRESHOLD = 0.25

In [63]:
[M, n, N] = [2000, 7, 12]
rv = hypergeom(M, n, N)
x = np.arange(0, n+1)
pmf_dogs = rv.pmf([7])
print(pmf_dogs[0])

3.151452666160824e-17


In [74]:
df_icd10_tree = pd.read_csv(ICD_TRANSALTOR_PATH)
icd10_translator_dict = df_icd10_tree[["coding","meaning"]].set_index("coding").to_dict("index")

In [17]:

extension = 'tsv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
print(result)

In [79]:
def get_hypogeomertic_pval(our_genes,their_genes):
    [M, n, N] = [18000, len(their_genes), (len(our_genes))]
    sampled = [len(our_genes.intersection(their_genes))]
    rv = hypergeom(M, n, N)
    x = np.arange(0, n+1)
    pmf_dogs = rv.pmf(sampled)
    return pmf_dogs[0]
results = []
for icd_val in result:
    symbol = icd_val.replace(".tsv","")
    val = symbol + "_1.csv"
    val = os.path.join(ICD_FILES_PATH,val)
    df_our = pd.read_csv(val)
    df_their = pd.read_csv(os.path.join(path,icd_val),sep="\t")
    our_genes = set(df_our[df_our["fdr_significance"]]["symbol"])
    their_genes_global = set(df_their[df_their["globalScore"] > GLOBAL_THREASHOLD]["symbol"])
    their_genes_genetics = set(df_their[df_their["otGeneticsPortal"].replace('No data',0).astype(float) > GENETICS_THRESHOLD]["symbol"])
    num_ours = len(our_genes)
    num_global = len(their_genes_global)
    num_genetics = len(their_genes_genetics)
    
    num_our_global = len(our_genes.intersection(their_genes_global))
    num_our_genetics = len(our_genes.intersection(their_genes_genetics))
    global_pval = get_hypogeomertic_pval(our_genes,their_genes_global)
    genetics_pval = get_hypogeomertic_pval(our_genes,their_genes_genetics)
    meaning = icd10_translator_dict[symbol]["meaning"]
    results.append([symbol,meaning,num_ours,num_global,num_genetics,num_our_global,num_our_genetics,global_pval,genetics_pval])
    

In [76]:
len(results[0])

9

In [80]:
df = pd.DataFrame(results,columns=["Phen","meaning","our","num_global","num_genetics","num_our_global","num_our_genetics","global_pval","genetics_pval"])
df.to_csv(f"/sci/nosnap/michall/roeizucker/jupyter_notebooks/PWAS_Hub/result_GLOBAL_THREASHOLD_{GLOBAL_THREASHOLD}_GENETICS_THRESHOLD_{GENETICS_THRESHOLD}.csv")

In [ ]:
# find icd10 files
SCORE_RANGE = 0.001
    # Walk through the directory
file_list = []
for root, dirs, files in os.walk(ICD_FILES_PATH):
    for file in files:
        # Check if the file ends with '_1.csv'
        if file.endswith('_1.csv'):
            # Add each matching file's full path to the list
            file_list.append(file.replace("_1.csv",""))
def create_dataframe_for_association_file(association,score_range,datatype=None):
    result = []
    for val in diseases:
        icd_vals = []
        for ref in (diseases[val]["dbXRefs"]):

            if "ICD10" in ref:
                icd_vals.append(ref.split(":")[1].replace(".",""))
        if len(icd_vals) == 1:
    #         check if there is an icd10 file that matches
            if icd_vals[0] in file_list and val in association:
                df = pd.read_csv(os.path.join(ICD_FILES_PATH,icd_vals[0] + "_1.csv"))
    #             print(df)
    #             print(icd_vals[0],val)
                raw_genes = association[val]
                gene_symbols = []
                for raw_gene in raw_genes:
                    
                    if raw_gene["score"] > score_range and (datatype is None or raw_gene["datatypeId"] == datatype):
                        gene_symbols.append(esng_translator[raw_gene["targetId"]]["name"])

                our_symbol = set(df[df["fdr_significance"]]["symbol"])
                their_symbol = set(gene_symbols)
                intersection = our_symbol.intersection(their_symbol)
                result.append([icd_vals[0] + "-" +val,len(intersection),len(our_symbol),len(their_symbol)])
    df = pd.DataFrame(result,columns=["Phen","intersection","our_genes","their_genes"])
    return df
#     df.to_csv("/sci/nosnap/michall/roeizucker/pwas_hub/opentargets_PWAS_intersection_Score_range" + str(SCORE_RANGE) + ".csv")   
df_genetic_association = create_dataframe_for_association_file(direct_by_source,SCORE_RANGE,"genetic_association")
df_general = create_dataframe_for_association_file(direct,SCORE_RANGE)
